In [39]:
import pandas as pd 
import numpy as np
import requests
import json
import time
import datetime as dt
import re
import os.path
from openpyxl import load_workbook
import os

import wget

from zipfile import ZipFile
import gzip
import os

In [40]:
Area_Code = {
'DZA':'Algeria',
'AGO':'Angola',
'BEN':'Benin',
'BWA':'Botswana',
'BFA':'Burkina Faso',
'BDI':'Burundi',
'CPV':'Cabo Verde',
'CMR':'Cameroon',
'CAF':'Central African Republic',
'TCD':'Chad',
'COM':'Comoros',
'COG':'Congo',
'CIV':'Côte d’Ivoire',
'COD':'Democratic Republic of the Congo',
'DJI':'Djibouti',
'EGY':'Egypt',
'GNQ':'Equatorial Guinea',
'ERI':'Eritrea',
'SWZ':'Eswatini',
'ETH':'Ethiopia',
'GAB':'Gabon',
'GMB':'Gambia',
'GHA':'Ghana',
'GIN':'Guinea',
'GNB':'Guinea-Bissau',
'KEN':'Kenya',
'LSO':'Lesotho',
'LBR':'Liberia',
'LBY':'Libya',
'MDG':'Madagascar',
'MWI':'Malawi',
'MLI':'Mali',
'MRT':'Mauritania',
'MUS':'Mauritius',
'MAR':'Morocco',
'MOZ':'Mozambique',
'NAM':'Namibia',
'NER':'Niger',
'NGA':'Nigeria',
'RWA':'Rwanda',
'SEN':'Senegal',
'SYC':'Seychelles',
'SLE':'Sierra Leone',
'SOM':'Somalia',
'ZAF':'South Africa',
'SSD':'South Sudan',
'STP':'So Tom & Prncipe',
'SDN':'Sudan',
'TGO':'Togo',
'TUN':'Tunisia ',
'UGA':'Uganda',
'TZA':'United Republic of Tanzania',
'ZMB':'Zambia',
'ZWE':'Zimbabwe',
}

select_code = [k for k in Area_Code.keys()]
# Getting year for citation
D = dt.datetime.today()
y = D.year

In [41]:
Indicators={
    'EMP_2WAP_SEX_AGE_RT':['Employment to population ratio','SOC.LBR.EMP'],
    'EMP_2WAP_SEX_AGE_GEO_RT':['Employment to population ratio','SOC.LBR.EMP'],
    
    'EAP_2WAP_SEX_AGE_RT': ['Labour force participation rate','SOC.LBR.PAR.RAT'],
    'EAP_2WAP_SEX_AGE_GEO_RT':['Labour force participation rate','SOC.LBR.PAR.RAT'],
    
    'UNE_2EAP_SEX_AGE_RT':['Unemployment rate','SOC.LBR.UER'],
    'UNE_2EAP_SEX_AGE_GEO_RT':['Unemployment rate','SOC.LBR.UER'],
    
    'UNE_3EAP_SEX_AGE_RT':['Youth unemployment rate','MDG1.8.2'],
    'UNE_3EAP_SEX_AGE_GEO_RT':['Youth unemployment rate','MDG1.8.2'],
    
    'EIP_2EIP_SEX_AGE_NB':['Population outside the labour force','ILO.OUT.LBR'],
    
    'EMP_2TRU_SEX_AGE_RT':['Time-related underemployment rate','LO.TR.UND.EMP.RAT'],
    
    'GDP_205U_NOC_NB':['Output per worker (GDP constant 2010 US $)','ILO.OUT.WKER.2010USD','CUSD','Constant USD'],
    
}
Ind_codes=list(Indicators.keys())

In [42]:
Indicators={
    'EIP_WDIS_SEX_AGE_NB':['Discouraged job-seekers by sex and age',''],
    'EES_TEES_SEX_EC2_NB':['Employees by age and economic activity - ISIC level 2',''],
    'EES_TEES_SEX_OC2_NB':['Employees by age and occupation - ISCO level 2',''],
    'EES_TEES_ECO_OCU_NB':['Employees by economic activity and occupation',''],
    'EES_TEES_SEX_EC2_NB':['Employees by sex and economic activity - ISIC level 2',''],
    'EES_TEES_SEX_ECO_NB':['Employees by sex and economic activity',''],
    'EES_TEES_SEX_INS_NB':['Employees by sex and institutional sector',''],
    'EES_TEES_SEX_OC2_NB':['Employees by sex and occupation - ISCO level 2',''],
    'EES_TEES_SEX_OCU_NB':['Employees by sex and occupation',''],
    'EES_TEES_SEX_AGE_JOB_NB':['Employees by sex, age and temporary or permanent contract',''],
    'EMP_TEMP_AGE_EC2_NB':['Employment by age and economic activity - ISIC level 2',''],
    'EMP_TEMP_AGE_OC2_NB':['Employment by age and occupation - ISCO level 2',''],
    'EMP_TEMP_ECO_OCU_NB':['Employment by economic activity and occupation',''],
    'EMP_2EMP_SEX_AGE_NB':['Employment by sex and age',''],
    'EMP_TEMP_SEX_EC2_NB':['Employment by sex and economic activity - ISIC level 2',''],
    'EMP_TEMP_SEX_ECO_NB':['Employment by sex and economic activity',''],
    'EMP_TEMP_SEX_INS_NB':['Employment by sex and institutional sector',''],
    'EMP_TEMP_SEX_MJH_NB':['Employment by sex and multiple-job holding',''],
    'EMP_TEMP_SEX_OC2_NB':['Employment by sex and occupation - ISCO level 2',''],
    'EMP_TEMP_SEX_OCU_NB':['Employment by sex and occupation',''],
    'EMP_2EMP_SEX_STE_NB':['Employment by sex and status in employment',''],
    'EMP_TEMP_SEX_AGE_ECO_NB':['Employment by sex, age and economic activity',''],
    'EMP_TEMP_SEX_AGE_OCU_NB':['Employment by sex, age and occupation',''],
    'EMP_TEMP_SEX_AGE_STE_NB':['Employment by sex, age and status in employment',''],
    'EMP_TEMP_SEX_ECO_EDU_NB':['Employment by sex, economic activity and education',''],
    'EMP_TEMP_SEX_OCU_INS_NB':['Employment by sex, occupation and institutional sector',''],
    'EMP_TEMP_SEX_STE_ECO_NB':['Employment by sex, status in employment and economic activity',''],
    'EMP_TEMP_SEX_STE_OCU_NB':['Employment by sex, status in employment and occupation',''],
    'EMP_PIFL_SEX_NB':['Employment outside the formal sector by sex',''],
    'EMP_PIFL_SEX_AGE_NB':['Employment outside the formal sector by sex and age',''],
    'EMP_PIFL_SEX_ECO_NB':['Employment outside the formal sector by sex and economic activity',''],
    'EMP_PIFL_SEX_STE_NB':['Employment outside the formal sector by sex and status in employment',''],
    'EMP_DWAP_SEX_AGE_RT':['Employment-to-population ratio by sex and age','SOC.LBR.EMP'],
    'EIP_DWAP_SEX_AGE_RT':['Inactivity rate by sex and age',''],
    'EMP_NIFL_SEX_NB':['Informal employment by sex',''],
    'EMP_NIFL_SEX_AGE_NB':['Informal employment by sex and age',''],
    'EMP_NIFL_SEX_ECO_NB':['Informal employment by sex and economic activity',''],
    'EMP_NIFL_SEX_STE_NB':['Informal employment by sex and status in employment',''],
    'EMP_NIFL_SEX_RT':['Informal employment rate by sex',''],
    'EMP_NIFL_SEX_AGE_RT':['Informal employment rate by sex and age',''],
    'EMP_NIFL_SEX_ECO_RT':['Informal employment rate by sex and economic activity',''],
    'EMP_NIFL_SEX_STE_RT':['Informal employment rate by status in employment (by sex)',''],
    'EAP_2EAP_SEX_AGE_NB':['Labour force by sex and age',''],
    'EAP_DWAP_SEX_AGE_RT':['Labour force participation rate by sex and age','SOC.LBR.PAR.RAT'],
    'HOW_TEMP_AGE_EC2_NB':['Mean weekly hours actually worked per employed person by age and economic activity - ISIC level 2 | Annual',''],
    'HOW_TEMP_AGE_OC2_NB':['Mean weekly hours actually worked per employed person by age and occupation - ISCO level 2 | Annual',''],
    'HOW_XEES_AGE_EC2_NB':['Mean weekly hours actually worked per employee by age and economic activity - ISIC level 2 | Annual',''],
    'HOW_XEES_AGE_OC2_NB':['Mean weekly hours actually worked per employee by age and occupation - ISCO level 2 | Annual',''],
    'EIP_TEIP_SEX_AGE_NB':['Persons outside the labour force by sex and age','ILO.OUT.LBR'],
    'EIP_2PLF_SEX_AGE_NB':['Potential labour force by sex and age',''],
    'PSE_TPSE_GOV_NB':['Public employment by sectors and sub-sectors of national accounts',''],
    'EMP_PIFL_SEX_RT':['Share of employment outside the formal sector by sex',''],
    'EMP_PIFL_SEX_AGE_RT':['Share of employment outside the formal sector by sex and age',''],
    'EMP_PIFL_SEX_ECO_RT':['Share of employment outside the formal sector by sex and economic activity',''],
    'EMP_PIFL_SEX_STE_RT':['Share of employment outside the formal sector by status in employment (by sex)',''],
    'EES_XTMP_SEX_RT':['Share of temporary employees by sex',''],
    'EIP_NEET_SEX_RT':['Share of youth not in employment, education or training (NEET) by sex',''],
    'EIP_NEET_SEX_AGE_RT':['Share of youth not in employment, education or training (NEET) by sex and age',''],
    'TRU_TTRU_SEX_AGE_NB':['Time-related underemployment by sex and age',''],
    'TRU_DEMP_SEX_AGE_RT':['Time-related underemployment rate by sex and age','LO.TR.UND.EMP.RAT'],
    'UNE_TUNE_SEX_AGE_NB':['Unemployment by sex and age',''],
    'UNE_TUNE_SEX_CAT_NB':['Unemployment by sex and categories of unemployed persons',''],
    'UNE_3UNE_SEX_AGE_DUR_NB':['Unemployment by sex, age and duration',''],
    'UNE_TUNE_SEX_ECO_NB':['Unemployment of previously employed persons by sex and former economic activity',''],
    'UNE_TUNE_SEX_OCU_NB':['Unemployment of previously employed persons by sex and former occupation',''],
    'UNE_TUNE_SEX_ECO_EDU_NB':['Unemployment of previously employed persons by sex, former economic activity and education',''],
    'UNE_DEAP_SEX_AGE_RT':['Unemployment rate by sex and age','SOC.LBR.UER'],
    'POP_XWAP_SEX_AGE_NB':['Working-age population by sex and age',''],
    'EIP_NEET_SEX_NB':['Youth not in employment, education or training (NEET) by sex',''],
    'EIP_NEET_SEX_AGE_NB':['Youth not in employment, education or training (NEET) by sex and age',''],
    'EIP_3DIS_SEX_AGE_GEO_NB':['Youth discouraged job-seekers by sex, age and rural / urban areas',''],
    'EES_3EES_SEX_AGE_JOB_NB':['Youth employees by sex, age and type of job contract',''],
    'EMP_3EMP_SEX_AGE_DSB_NB':['Youth employment by sex, age and disability status',''],
    'EMP_3EMP_SEX_AGE_ECO_NB':['Youth employment by sex, age and economic activity',''],
    'EMP_3EMP_SEX_AGE_EDU_NB':['Youth employment by sex, age and education',''],
    'EMP_3EMP_SEX_AGE_OCU_NB':['Youth employment by sex, age and occupation',''],
    'EMP_3EMP_SEX_AGE_GEO_NB':['Youth employment by sex, age and rural / urban areas',''],
    'EMP_3EMP_SEX_AGE_STU_NB':['Youth employment by sex, age and school attendance status',''],
    'EMP_3EMP_SEX_AGE_STE_NB':['Youth employment by sex, age and status in employment',''],
    'EMP_3EMP_SEX_AGE_HOW_NB':['Youth employment by sex, age and weekly hours actually worked',''],
    'EMP_3EMP_SEX_AGE_JOB_NB':['Youth employment by sex, age and working time arrangement',''],
    'EMP_3WAP_SEX_AGE_DSB_RT':['Youth employment-to-population ratio by sex, age and disability status',''],
    'EMP_3WAP_SEX_AGE_EDU_RT':['Youth employment-to-population ratio by sex, age and education',''],
    'EMP_3WAP_SEX_AGE_GEO_RT':['Youth employment-to-population ratio by sex, age and rural / urban areas',''],
    'EMP_3WAP_SEX_AGE_STU_RT':['Youth employment-to-population ratio by sex, age and school attendance status',''],
    'EIP_3WAP_SEX_AGE_DSB_RT':['Youth inactivity rate by sex, age and disability status',''],
    'EIP_3WAP_SEX_AGE_EDU_RT':['Youth inactivity rate by sex, age and education',''],
    'EIP_3WAP_SEX_AGE_GEO_RT':['Youth inactivity rate by sex, age and rural / urban areas',''],
    'EIP_3WAP_SEX_AGE_STU_RT':['Youth inactivity rate by sex, age and school attendance status',''],
    'EAP_3EAP_SEX_AGE_DSB_NB':['Youth labour force by sex, age and disability status',''],
    'EAP_3EAP_SEX_AGE_EDU_NB':['Youth labour force by sex, age and education',''],
    'EAP_3EAP_SEX_AGE_GEO_NB':['Youth labour force by sex, age and rural / urban areas',''],
    'EAP_3EAP_SEX_AGE_STU_NB':['Youth labour force by sex, age and school attendance status',''],
    'EAP_3WAP_SEX_AGE_DSB_RT':['Youth labour force participation rate by sex, age and disability status',''],
    'EAP_3WAP_SEX_AGE_EDU_RT':['Youth labour force participation rate by sex, age and education',''],
    'EAP_3WAP_SEX_AGE_GEO_RT':['Youth labour force participation rate by sex, age and rural / urban areas',''],
    'EAP_3WAP_SEX_AGE_STU_RT':['Youth labour force participation rate by sex, age and school attendance status',''],
    'EIP_3EIP_SEX_AGE_DSB_NB':['Youth outside the labour force by sex, age and disability status',''],
    'EIP_3EIP_SEX_AGE_EDU_NB':['Youth outside the labour force by sex, age and education',''],
    'EIP_3EIP_SEX_AGE_GEO_NB':['Youth outside the labour force by sex, age and rural / urban areas',''],
    'EIP_3EIP_SEX_AGE_STU_NB':['Youth outside the labour force by sex, age and school attendance status',''],
    'TRU_3TRU_SEX_AGE_GEO_NB':['Youth time-related underemployment by sex, age and rural / urban areas',''],
    'POP_3TED_SEX_ECO_NB':['Youth transited by sex and economic activity',''],
    'POP_3TED_SEX_OCU_NB':['Youth transited by sex and occupation',''],
    'POP_3TED_SEX_STE_NB':['Youth transited by sex and status in employment',''],
    'UNE_3UNE_SEX_AGE_CAT_NB':['Youth unemployment by sex, age and categories of unemployed persons',''],
    'EIP_3EIP_SEX_AGE_DSB_NB':['Youth unemployment by sex, age and disability status',''],
    'UNE_3UNE_SEX_AGE_DUR_NB':['Youth unemployment by sex, age and duration',''],
    'UNE_3UNE_SEX_AGE_EDU_NB':['Youth unemployment by sex, age and education',''],
    'UNE_3UNE_SEX_AGE_GEO_NB':['Youth unemployment by sex, age and rural / urban areas',''],
    'UNE_3UNE_SEX_AGE_STU_NB':['Youth unemployment by sex, age and school attendance status',''],
    'UNE_3EAP_SEX_AGE_DSB_RT':['Youth unemployment rate by sex, age and disability status','MDG1.8.2'],
    'UNE_3EAP_SEX_AGE_EDU_RT':['Youth unemployment rate by sex, age and education','MDG1.8.2'],
    'UNE_3EAP_SEX_AGE_GEO_RT':['Youth unemployment rate by sex, age and rural / urban areas','MDG1.8.2'],
    'UNE_3EAP_SEX_AGE_STU_RT':['Youth unemployment rate by sex, age and school attendance status','MDG1.8.2'],
    'UNE_3WAP_SEX_AGE_STU_RT':['Youth unemployment-to-population ratio by sex, age and school attendance status',''],
    'POP_3WAP_SEX_AGE_DSB_NB':['Youth working-age population by sex, age and disability status',''],
    'POP_3WAP_SEX_AGE_EDU_NB':['Youth working-age population by sex, age and education',''],
    'POP_3FOR_SEX_AGE_TRA_NB':['Youth working-age population by sex, age and forms of transition',''],
    'POP_3WAP_SEX_AGE_LMS_NB':['Youth working-age population by sex, age and labour market status',''],
    'POP_3WAP_SEX_AGE_GEO_NB':['Youth working-age population by sex, age and rural / urban areas',''],
    'POP_3WAP_SEX_AGE_STU_NB':['Youth working-age population by sex, age and school attendance status',''],
    'POP_3STG_SEX_AGE_TRA_NB':['Youth working-age population by sex, age and stages of transition',''],
    'POP_3FOR_SEX_EDU_TRA_NB':['Youth working-age population by sex, education and forms of transition',''],
    'POP_3STG_SEX_EDU_TRA_NB':['Youth working-age population by sex, education and stages of transition',''],
    'POP_3FOR_SEX_GEO_TRA_NB':['Youth working-age population by sex, rural / urban areas and forms of transition',''],
    'POP_3STG_SEX_GEO_TRA_NB':['Youth working-age population by sex, rural / urban areas and stages of transition',''],
}
Ind_codes=list(Indicators.keys())
len(Ind_codes)

123

In [43]:
Dim_age={
    'YGE15':['15+','15+'],
    'Y15-24':['youth','YTH'],
    'YGE25': ['25+','25+'],
    'Y15-29':['15-29','15-29'],
    'Y15-19':['15-19','15-19'],
    'Y20-24': ['20-24','20-24',],
    'Y25-29':['25-29','25-29'],
    'Y15-64':['15-64','15-64'],
    'TOTAL':['Total','TL'],
    'Y25-54':['25-54','25-54'],
    'Y55-64':['55-64','55-64'],
    'YGE65':['65+','65+'],
    'Y25-34':['25-34','25-34'],
    'Y35-44':['35-44','35-44'],
    'Y45-54':['45-54','45-54'],
    'Y30-34':['30-34','30-34'],
    'Y35-39':['35-39','35-39'],
    'Y40-44':['40-44','40-44'],
    'Y45-49':['45-49','45-49'],
    'Y50-54':['50-54','50-54'],
    'Y55-59':['55-59','55-59'],
    'Y60-64':['60-64','60-64'],
    'Y10-14':['10-14','10-14'],
    'YLT15':['<15','<15'],
    'Y05-09':['05-09','05-09'],
}
Dim_sex = {
    'T':['Total','TL'],
    'F':['Female','PF'],
    'M':['Male','PM'],
}
Dim_geo={
    'NAT': ['National','NAT'],
    'RUR': ['Rural','RUR'],
    'URB': ['Urban','URB'],
    'X':['Not elsewhere classified','X'],
}

In [44]:
Unit={
    'RT':['%','PERC'],
    'NB':['number','NSO'],
}

In [45]:
cols = ['Areacode', 'IndicatorName', 'Areaenglish', 'DataSource',
       'DataProviderId', 'SpecificSource', 'Indicatorcode', 'Modecode',
       'Unitcode', 'Year', 'Value']
DataFrame = pd.DataFrame(columns = cols)

In [ ]:
for A_code in select_code[37:38]:
    url = f'https://www.ilo.org/ilostat-files/WEB_bulk_download/ref_area/{A_code}_A.csv.gz'
    wget.download(url)

    f_name = f'{A_code}_A.csv.gz'

    df = pd.read_csv(f_name, compression='gzip', engine='python')
    os.remove(f'{A_code}_A.csv.gz') 

    df= df[['ref_area','indicator','sex','classif1','classif2','time','obs_value']]
    print(f'Data for {country_name} Loading!')
        #Selecting the inidicators for ECA
    Data = df.loc[df['indicator'].isin(Ind_codes)]
    Data = Data.reset_index(drop=True)
    for i in range(len(Data)):
        ILO_code = Data.loc[i]['indicator']
        ILO_code_split = ILO_code.split('_')
        atributes = ILO_code_split[2:-1]
        unit = Unit[ILO_code_split[-1]]

        name = Indicators[ILO_code][0]
        code = Indicators[ILO_code][1] 
        print(ILO_code)
        mode=''
        if 'AGE' in atributes:
            if not pd.isna(Data.loc[i]['classif1']):
                C_1 = Data.loc[i]['classif1'].split('_')[-1]
                n = Dim_age[C_1][0]
                c = Dim_age[C_1][1]
                name +=f', {n}'
                code += f'.{c}'
        if 'GEO' in atributes and 'COV' in atributes:
            if not pd.isna(Data.loc[i]['classif2']):
                
                C_2 = Data.loc[i]['classif2'].split('_')[-1]
                
                if Dim_geo[C_2][1]=="NAT":
                    pass
                else:
                    name += f' - {Dim_geo[C_2][0]}'
                    code += f'.{Dim_geo[C_2][1]}'
        if 'SEX' in atributes:
            sex_at = Data.loc[i]['sex'].split('_')[-1]  # T,or F or M
            print(Data.loc[i]['sex'])
            print(sex_at)
            name += f' - {Dim_sex[sex_at][0]}'
            mode = Dim_sex[sex_at][1]

        name +=f' ({unit[0]})'

        Data.iloc[i,1] = name
        Data.iloc[i,2] = code
        Data.iloc[i,3] = mode
        Data.iloc[i,4] = unit[1]



    Data.rename(columns = {'ref_area':'Areacode','indicator':'IndicatorName',\
                   'sex':'Indicatorcode','classif1':'Modecode',\
                   'classif2':'Unitcode','time':'Year','obs_value':'Value'}, inplace = True)

    # Data Source
    DataSource='International Labour Organization (ILO)'
    # Data Provider ID
    DataProviderID=11

    country_name = Area_Code[A_code]
    cite = f'Retrived from: {url}, On: {str(D.date())}, By web scraping using the python libraries: requests and pandas'
    SpecificSource = [cite]*len(Data)
    Data.insert(7, 'SpecificSource', SpecificSource )
    Data.insert(7, 'DataProviderId', [DataProviderID]*len(Data) )
    Data.insert(7, 'DataSource', [DataSource]*len(Data) )
    Data.insert(0, 'Areaenglish', [country_name]*len(Data) )
    Data.to_excel(f'Data/{country_name}.xlsx', index=False)
    DataFrame = pd.concat([DataFrame, Data], ignore_index=True, sort=False)
    print(f'Data for {country_name} Done!')
    
DataFrame.to_excel(f'All.xlsx', index=False)
print("All Done")